In [65]:
import requests
from BeautifulSoup import BeautifulSoup
import pandas as pd
from hashlib import md5

In [72]:
tmpfile = '/tmp/tmpfile.txt'

In [91]:
def getDataList(page):
    # 读取文件
    f = open(page)
    content = ''.join(f.readlines())
    # 读取url列表
    soup = BeautifulSoup(content)
    table = soup.find('tbody')
    trs = table.findAll('tr')
    # 迭代处理
    data = []
    for tr in trs:
        # 读取f1
        td = tr.find('td')
        f1 = float(td.getText())
        # 读取url 
        a = tr.find('a')
        url =  a.attrs[0][1]
        data.append({'f1': f1, 'url': url})
    return data

In [ ]:
def downloadFile(url, saveto):
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(saveto, 'wb') as f:
            for chunk in r:
                f.write(chunk)

In [100]:
def extractData(page):
    print u'正在处理:%s ... ' % page,
    dataList = getDataList(page)
    for data in dataList:
        downloadFile(data['url'], tmpfile)
        filename = 'commits/%s.txt' % md5(data['url']).hexdigest()
        df = pd.read_csv(tmpfile, sep='\t', names=['dev_id', 'post_id', 'p'])
        df['f1'] =  data['f1']
        df = df.dropna(how='any')
        df.to_csv(filename, index=False)
    print u'(ok)'

In [101]:
page = 'pages/1.html'

In [102]:
extractData(page)

正在处理:pages/1.html ...  (ok)
